# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [5]:
!jupyter nbextension enable --py --sys-prefix gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [6]:
weather_df = pd.read_csv("../output_data/cities.csv")
weather_df = weather_df.drop(weather_df.columns[0], axis=1)

weather_df



,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...
543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
locations = weather_df[["Lat", "Lng"]].astype(float)
humidity = weather_df["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
hoteldf = weather_df.loc[(weather_df['Max Temp'] >= 70) & (weather_df['Max Temp'] <= 80) & (weather_df['Cloudiness']==0) & (weather_df['Wind Speed'] < 10)]
hoteldf

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44
70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hoteldf["Hotel Name"]=''
hoteldf

<ipython-input-9-49cdbda61677>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hoteldf["Hotel Name"]=''


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44,
70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,
88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,
138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,
139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,
176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,
297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,
444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17,


In [18]:
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": g_key
}


for index, row in hoteldf.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]

    
    params["location"] = f"{lat},{lng}"

   
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    
    hotel_lists = requests.get(base_url, params=params)
    

    hotel_lists = hotel_lists.json()
    
   
    try:
        hoteldf.loc[index, "Hotel Name"] = hotel_lists["results"][0]["name"]
       
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
hotel_lists

C:\Users\Cemen\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Missing field/result... skipping.


{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 32.8802023, 'lng': 59.216279},
    'viewport': {'northeast': {'lat': 32.88151937989272,
      'lng': 59.21771057989272},
     'southwest': {'lat': 32.87881972010727, 'lng': 59.21501092010728}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'name': 'هتل پرويز',
   'place_id': 'ChIJ8dNk011nGj8RZI51wiSCBsY',
   'plus_code': {'compound_code': 'V6J8+3G Birjand, South Khorasan Province, Iran',
    'global_code': '8H4XV6J8+3G'},
   'rating': 3,
   'reference': 'ChIJ8dNk011nGj8RZI51wiSCBsY',
   'scope': 'GOOGLE',
   'types': ['lodging', 'point_of_interest', 'establishment'],
   'user_ratings_total': 1,
   'vicinity': 'Birjand'},
  {'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 32.8851325, 'lng': 59.2167535},
    'viewport': {'northeast': {'lat': 32.88649142989272,
      'lng': 59.21818162989271},
     'southwest': {

In [17]:
hoteldf

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44,Opuwo Country Lodge
70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,Pousada Suítes do Lago
88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,Hotel Castelo
138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,MAEVA HOTEL
139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,
176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,Hotel Mediterranee
297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,Mount Mogok Hotel
444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17,هتل پرويز


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hoteldf.iterrows()]
locations = hoteldf[["Lat", "Lng"]]

In [15]:
locations = weather_df[["Lat", "Lng"]].astype(float)
humidity = weather_df["Humidity"].astype(float)

gmaps.configure(api_key=g_key)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)

hotel_locations = hoteldf[["Lat", "Lng"]].astype(float)

markers = gmaps.marker_layer(hotel_locations,info_box_content=hotel_info)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# Add marker layer ontop of heat map


# Display figure
